In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
import pickle
from collections import OrderedDict
from hyperopt import fmin, tpe, hp, Trials, space_eval, plotting
from tflite2xcore.model_generation import utils
import tensorflow as tf
from sklearn.externals import joblib
utils.set_all_seeds(42)

In [2]:
SEARCH_SPACE = OrderedDict([('learning_rate',
                             hp.loguniform('learning_rate', np.log(0.01), np.log(0.5))),
                            ('epochs',
                             hp.choice('epochs', range(1, 51, 1))),
                            ('batch_size',
                             hp.choice('batch_size', [32, 64, 128, 256, 512])),
                            ('l1_reg',
                             hp.choice('l1_reg', np.arange(1e-5, 2e-4, 1e-6)))
                           ])
STATIC_PARAMS = {'boosting': 'gbdt',
                'objective':'binary',
                'metric': 'auc',
                'num_threads': 12,
                }
# Data
data = utils.prepare_MNIST(False, simard=False, padding=0)

for k, v in data.items():
    print(f"Prepped data[{k}] with shape: {v.shape}")

Prepped data[x_train] with shape: (4096, 28, 28, 1)
Prepped data[x_test] with shape: (1024, 28, 28, 1)
Prepped data[x_val] with shape: (100, 28, 28, 1)
Prepped data[y_train] with shape: (4096,)
Prepped data[y_test] with shape: (1024,)
Prepped data[y_val] with shape: (100,)


In [3]:
def train_evaluate(params):
    core_model = tf.keras.Sequential(
        name='logistic_regression',
        layers=[
            tf.keras.layers.Flatten(input_shape=(28, 28, 1), name='input'),
            tf.keras.layers.Dense(10,
                                  activation='softmax',
                                  kernel_regularizer=tf.keras.regularizers.l1(params['l1_reg']))
        ]
    )
    core_model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=params['learning_rate']),
        metrics=['accuracy'])
    core_model.fit(
        data['x_train'], data['y_train'],
        validation_data=(data['x_test'], data['y_test']),
        batch_size=params['batch_size'],
        verbose=0,
        epochs=params['epochs']
    )
    _, accuracy = core_model.evaluate(data['x_test'], data['y_test'])
    return accuracy

def objective(params):
    all_params = {**params, **STATIC_PARAMS}
    return -1.0 * train_evaluate(all_params)

In [4]:
trials = Trials()
HPO_PARAMS = {'max_evals': 1000,
              'trials': trials
             }
best = fmin(
    fn=objective,
    space=SEARCH_SPACE,
    algo=tpe.rand.suggest,
    **HPO_PARAMS
)

  32/1024 [..............................]              
 - ETA: 0s - loss: 0.5641 - accuracy: 0.9062            
                                                       
1024/1024 [==============================]              
 - 0s 23us/sample - loss: 0.7289 - accuracy: 0.8662     

  32/1024 [..............................]                                     
 - ETA: 1s - loss: 1.4223 - accuracy: 0.8438                                   
                                                                              
1024/1024 [==============================]                                     
 - 0s 60us/sample - loss: 2.3980 - accuracy: 0.8223                            

  32/1024 [..............................]                                     
 - ETA: 1s - loss: 0.3555 - accuracy: 0.9375                                   
                                                                              
1024/1024 [==============================]                                  

In [7]:
print(f"Found minimum after {HPO_PARAMS['max_evals']} trials:")
print(best)
print("")
print(space_eval(SEARCH_SPACE, best))

Found minimum after 1000 trials:
{'batch_size': 2, 'epochs': 8, 'l1_reg': 12, 'learning_rate': 0.010578650580041293}

{'batch_size': 128, 'epochs': 9, 'l1_reg': 2.2000000000000006e-05, 'learning_rate': 0.010578650580041293}


In [ ]:
best_params = space_eval(SEARCH_SPACE, best)
objective(best_params)

In [ ]:
l = [-objective(best_params) for e in range(50)]
maxv = np.max(l)
minv = np.min(l)
print(f"Acc range: ({maxv}, {minv})\nAcc median: {np.median(l)}\nAcc average: {np.average(l)}")
from hyperopt import plotting
plotting.main_plot_history(trials)
plotting.main_plot_histogram(trials)

In [ ]:
joblib.dump(trials, './tune_logistic_regression_trials.pkl')

In [ ]:
trials = joblib.load('./tune_logistic_regression_trials.pkl')

In [ ]:
from hyperopt import plotting
plotting.main_plot_history(trials)
plotting.main_plot_histogram(trials)